In [25]:
# basic operations
import numpy as np
import pandas as pd 

In [26]:
Eng_df=pd.read_csv('EP.csv')

In [27]:
Eng_df = Eng_df.dropna()
data = Eng_df
data

,Div,Date,Time,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,5/8/2022,20:00,Crystal Palace,Arsenal,0.0,2.0,A,0.0,1.0,...,1.76,0.50,2.09,1.84,2.04,1.88,2.09,1.88,2.03,1.85
1,E0,6/8/2022,12:30,Fulham,Liverpool,2.0,2.0,D,1.0,0.0,...,2.73,1.75,1.90,2.03,1.91,2.02,2.01,2.06,1.89,1.99
2,E0,6/8/2022,15:00,Bournemouth,Aston Villa,2.0,0.0,H,1.0,0.0,...,1.76,0.50,1.93,2.00,1.93,2.00,1.94,2.04,1.88,2.00
3,E0,6/8/2022,15:00,Leeds,Wolves,2.0,1.0,H,1.0,1.0,...,1.87,-0.25,2.08,1.85,2.10,1.84,2.14,1.87,2.08,1.81
4,E0,6/8/2022,15:00,Newcastle,Nott'm Forest,2.0,0.0,H,0.0,0.0,...,1.89,-1.00,1.97,1.96,1.99,1.93,2.19,1.97,2.03,1.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1421,E0,26/07/2020,16:00,Leicester,Man United,0.0,2.0,A,0.0,0.0,...,2.03,0.50,1.89,2.01,1.88,2.05,1.94,2.05,1.86,2.02
1422,E0,26/07/2020,16:00,Man City,Norwich,5.0,0.0,H,2.0,0.0,...,5.77,-3.75,2.03,1.87,2.01,1.88,2.06,1.88,2.02,1.84
1423,E0,26/07/2020,16:00,Newcastle,Liverpool,1.0,3.0,A,1.0,1.0,...,2.40,1.00,1.94,1.96,1.95,1.97,2.03,2.00,1.95,1.92
1424,E0,26/07/2020,16:00,Southampton,Sheffield United,3.0,1.0,H,0.0,1.0,...,2.01,-0.50,1.97,1.93,2.00,1.93,2.03,1.96,1.98,1.89


In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from time import time
from sklearn.metrics import f1_score
from os import path, makedirs, walk
from joblib import dump, load
import json

In [29]:
def train_classifier(clf, X_train, y_train):
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    print("Model trained in {:2f} seconds".format(end-start))


def predict_labels(clf, features, target):
    start = time()
    y_pred = clf.predict(features)
    end = time()
    print("Made Predictions in {:2f} seconds".format(end-start))

    acc = sum(target == y_pred) / float(len(y_pred))

    return f1_score(target, y_pred, average='micro'), acc


def model(clf, X_train, y_train, X_test, y_test):
    train_classifier(clf, X_train, y_train)

    f1, acc = predict_labels(clf, X_train, y_train)
    print("Training Info:")
    print("-" * 20)
    print("F1 Score:{}".format(f1))
    print("Accuracy:{}".format(acc*100))

    f1, acc = predict_labels(clf, X_test, y_test)
    print("Test Metrics:")
    print("-" * 20)
    print("F1 Score:{}".format(f1))
    print("Accuracy:{}".format(acc*100))


def derive_clean_sheet(src):
    arr = []
    n_rows = src.shape[0]

    for data in range(n_rows):

        #[HTHG, HTAG]
        values = src.iloc[data].values
        cs = [0, 0]

        if values[0] == 0:
            cs[1] = 1

        if values[1] == 0:
            cs[0] = 1

        arr.append(cs)

    return arr

In [30]:
# Pre processing

input_filter = ['home_encoded', 'away_encoded', 'HTHG', 'HTAG', 'HS',
                'AS', 'HST', 'AST', 'HR', 'AR']
output_filter = ['FTR']

cols_to_consider = input_filter + output_filter

encoder = LabelEncoder()
home_encoded = encoder.fit_transform(data['HomeTeam'])
home_encoded_mapping = dict(
    zip(encoder.classes_, encoder.transform(encoder.classes_).tolist()))
data['home_encoded'] = home_encoded

encoder = LabelEncoder()
away_encoded = encoder.fit_transform(data['AwayTeam'])
away_encoded_mapping = dict(
    zip(encoder.classes_, encoder.transform(encoder.classes_).tolist()))
data['away_encoded'] = away_encoded


In [31]:
data = data[cols_to_consider]

print(data[data.isna().any(axis=1)])
data = data.dropna(axis=0)


Empty DataFrame
Columns: [home_encoded, away_encoded, HTHG, HTAG, HS, AS, HST, AST, HR, AR, FTR]
Index: []


In [32]:
X = data[input_filter]
Y = data['FTR']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

svc_classifier = SVC(random_state=100, kernel='rbf')
lr_classifier = LogisticRegression(multi_class='ovr', max_iter=500)
nbClassifier = GaussianNB()
dtClassifier = DecisionTreeClassifier()
rfClassifier = RandomForestClassifier()

# print("Support Vector Machine")
# print("-" * 20)
# model(svc_classifier, X_train, Y_train, X_test, Y_test)

print()
print("Logistic Regression one vs All Classifier")
print("-" * 20)
model(lr_classifier, X_train, Y_train, X_test, Y_test)

print()
print("Gaussain Naive Bayes Classifier")
print("-" * 20)
model(nbClassifier, X_train, Y_train, X_test, Y_test)


Logistic Regression one vs All Classifier
--------------------
Model trained in 0.098399 seconds
Made Predictions in 0.002001 seconds
Training Info:
--------------------
F1 Score:0.6804577464788732
Accuracy:68.04577464788733
Made Predictions in 0.002002 seconds
Test Metrics:
--------------------
F1 Score:0.6373239436619719
Accuracy:63.732394366197184

Gaussain Naive Bayes Classifier
--------------------
Model trained in 0.006503 seconds
Made Predictions in 0.001000 seconds
Training Info:
--------------------
F1 Score:0.6311619718309859
Accuracy:63.116197183098585
Made Predictions in 0.003002 seconds
Test Metrics:
--------------------
F1 Score:0.602112676056338
Accuracy:60.2112676056338


In [33]:
print()
print("Random Forest Classifier")
print("-" * 20)
model(rfClassifier, X_train, Y_train, X_test, Y_test)

# Exporting the Model
print()
print()
shouldExport = input('Do you want to export the model(s) (y / n) ? ')
if shouldExport.strip().lower() == 'y':
    exportedModelsPath = 'exportedModels'

    makedirs(exportedModelsPath, exist_ok=True)

    dump(lr_classifier, f'{exportedModelsPath}/lr_classifier.model')
    dump(nbClassifier, f'{exportedModelsPath}/nb_classifier.model')
    dump(rfClassifier, f'{exportedModelsPath}/rf_classifier.model')

    exportMetaData = dict()
    exportMetaData['home_teams'] = home_encoded_mapping
    exportMetaData['away_teams'] = away_encoded_mapping

    exportMetaDataFile = open(f'{exportedModelsPath}/metaData.json', 'w')
    json.dump(exportMetaData, exportMetaDataFile)

    print(f'Model(s) exported successfully to {exportedModelsPath}/')


Random Forest Classifier
--------------------
Model trained in 0.274164 seconds
Made Predictions in 0.029556 seconds
Training Info:
--------------------
F1 Score:1.0
Accuracy:100.0
Made Predictions in 0.018535 seconds
Test Metrics:
--------------------
F1 Score:0.6338028169014085
Accuracy:63.38028169014085


Do you want to export the model(s) (y / n) ? n
